<div style="background-color: #000000;" >
<h1 style="margin: auto; padding: 20px; color:#fff; ">La Poule Qui Chante</h1>
</div>

<div style="background-color: #000000;" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Contexte et mission</h2>
</div>

La poule qui chante, une entreprise française d’agroalimentaire dont l'activité principale est l’élevage et la vente de poulets sous le label “Poulet Agriculture Biologique” souhaite évaluer la possibilité de se développer à l'international sous l'impulsion de son PDG. 

Pour l'instant, le champ des possibles est bien large : aucun pays particulier ni aucun continent n'est pour le moment choisi. Tous les pays sont envisageables !

L'objectif sera de proposer une analyse des groupements de pays que l’on peut cibler pour exporter nos poulets. 
Nous approfondirons ensuite l'étude de marché. 

 

Tu seras en autonomie sur ce projet, notamment sur le choix des données à analyser et du langage (R ou Python). 

Pars des données de la FAO (Food and Agriculture Organization) en pièce jointe pour commencer ton analyse puis : 
utilise l’analyse PESTEL pour trouver des idées de nouvelles données à ajouter (nous voulons au minimum 8 variables) ;
récupère et utilise toutes les données en open data que tu souhaites sur le site de la FAO, de la banque mondiale ou encore sur données mondiales.

**Prépare et nettoie les données :** 
Si tu utilises plusieurs sources de données, regroupe-les dans un même fichier. 
L’idéal serait d’avoir au minimum 100 pays dans notre analyse (qui couvrent au moins 60% de la population mondiale). 
Passe ensuite à l’exploration des données (en Python ou en R).

<div style="background-color: #000000;" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 1 - Importation des librairies et chargement des fichiers</h2>
</div>

<h3 style="margin: auto; padding: 20px; color: #000000; ">1.1 - Importation des librairies</h3>
</div>

In [185]:
# Importation des librairies

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
import scientisttools as sct
from tabulate import tabulate
from fuzzywuzzy import process
from pathlib import Path
from unidecode import unidecode

In [186]:
# Afficher toutes les colonnes d'un dataframe
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

<h3 style="margin: auto; padding: 20px; color: #000000; ">1.2 - Chargements des fichiers</h3>
</div>

In [187]:
from pathlib import Path
import pandas as pd

# Chemin vers les fichiers
path = Path(
    r"C:\Users\jerem\OneDrive\EMPLOI\PôleEmploi\Formation\Data Analyst\Openclassroom\Mes projets\Projet 11\LaPouleQuiChante"
)
files = list(path.glob("*.csv"))

# Chargement des fichiers dans un dictionnaire
dataframes_dict = {}
for file in files:
    for sep in [";", ",", "\t"]:
        try:
            df = pd.read_csv(file, sep=sep, encoding="utf-8")
            # Utiliser le nom du fichier sans extension comme clé
            dataframes_dict[file.stem] = df
            print(f"Lecture réussie pour {file.name} avec le séparateur '{sep}'")
            break
        except (UnicodeDecodeError, pd.errors.ParserError):
            continue
    else:
        print(f"Impossible de lire le fichier {file.name}.")

# Afficher les noms des fichiers chargés
print("\n--- Fichiers chargés ---")
for name in dataframes_dict.keys():
    print(f"- {name}")

Lecture réussie pour correspondance_pays-DESKTOP-7JGGVM9.csv avec le séparateur ';'
Lecture réussie pour correspondance_pays.csv avec le séparateur ';'
Lecture réussie pour correspondance_pays_2.csv avec le séparateur ';'
Lecture réussie pour couts_procedure_exportation_formalite_douaniere_usd.csv avec le séparateur ';'
Lecture réussie pour df_poule_qui_chante.csv avec le séparateur ';'
Lecture réussie pour DisponibiliteAlimentaire_2017.csv avec le séparateur ';'
Lecture réussie pour jointure_8.csv avec le séparateur ';'
Lecture réussie pour pib_par_habitant_ppa.csv avec le séparateur ';'
Lecture réussie pour Population_2000_2018.csv avec le séparateur ';'
Lecture réussie pour stabilite_politique_2017.csv avec le séparateur ';'
Lecture réussie pour taux_de_change_standardise.csv avec le séparateur ';'
Lecture réussie pour terre_agricole_territoire_%.csv avec le séparateur ';'

--- Fichiers chargés ---
- correspondance_pays-DESKTOP-7JGGVM9
- correspondance_pays
- correspondance_pays_2
-

<h3 style="margin: auto; padding: 20px; color: #000000; ">1.3 - Chargements des fonctions</h3>
</div>

In [188]:
# Création de la fonction description variables ou dataset


def description_variables(df):
    # Afficher le type de la variable
    type_variable = df.dtypes

    # Nombre de lignes totales
    nb_lignes = len(df.index)

    # Nombre total de valeurs dans chaque colonne
    total_values = df.count()

    # Nombre de valeurs uniques dans chaque colonnes
    valeurs_uniques = df.nunique()

    # Calculer le nombre de valeurs manquantes pour chaque colonne
    missing_counts = df.isnull().sum()

    # Calculer le pourcentage de valeurs manquantes pour chaque colonne
    missing_percentage = (missing_counts / df.shape[0] * 100).round(2)

    # Créer un DataFrame avec les résultats
    variables_infos = pd.DataFrame(
        {
            "type": type_variable,
            "total": nb_lignes,
            "n": total_values,
            "unique": valeurs_uniques,
            #'doublons' : nb_doublons,
            "NaN": missing_counts,
            "pourcentage_(NaN)": missing_percentage,
        }
    )

    # Trier par 'pourcentage_(NaN)' dans l'ordre décroissant
    variables_infos = variables_infos.sort_values(
        by="pourcentage_(NaN)", ascending=False
    )

    print("Description donnée(s):")
    print("Le jeu de données comporte: {} observation(s)".format(df.shape[0]))
    print("Le jeu de données comporte: {} colonne(s)".format(df.shape[1]))

    return variables_infos

In [189]:
# Fonction pour les variables quantitatives


def statistiques_descriptives(data):

    # Vérifier si l'argument est une Series ou un DataFrame

    if isinstance(data, pd.Series):

        df = data.to_frame()

    elif isinstance(data, pd.DataFrame):

        df = data

    else:

        raise TypeError("L'argument doit être une Series ou un DataFrame de pandas")

    # Sélectionner seulement les colonnes numériques

    df_numeric = df.select_dtypes(include="number")

    # Nombre total de valeurs dans chaque colonne

    valeurs_totales = df_numeric.count()

    # Nombre de valeurs uniques dans chaque colonne

    valeurs_uniques = df_numeric.nunique()

    # Valeur minimum dans chaque colonne

    minimum = df_numeric.min()

    # Moyenne dans chaque colonne

    moyenne = df_numeric.mean()

    # 1er quantile dans chaque colonne

    q1 = df_numeric.quantile(0.25)

    # Mediane dans chaque colonne

    mediane = df_numeric.median()

    # 3e quantile dans chaque colonne

    q3 = df_numeric.quantile(0.75)

    # Calcul de l'intervalle interquartile - IQR

    iqr = q3 - q1

    # Valeur maximum dans chaque colonne

    maximum = df_numeric.max()

    # Ecart-type dans chaque colonne

    std = df_numeric.std()

    # Variance dans chaque colonne

    variance = df_numeric.var()

    # Skewness dans chaque colonne

    skewness = df_numeric.skew()

    # Kurtosis dans chaque colonne

    kurtosis = df_numeric.kurtosis()

    # Créer un DataFrame avec les résultats

    stats_descriptives = pd.DataFrame(
        {
            "n": valeurs_totales,
            "unique": valeurs_uniques,
            "min": minimum,
            "max": maximum,
            "moy": moyenne,
            "mediane": mediane,
            "q1": q1,
            "iqr": iqr,
            "q3": q3,
            "std": std,
            "var": variance,
            "skew": skewness,
            "kurt": kurtosis,
        }
    )

    print("Statistiques descriptives:")

    return stats_descriptives

In [190]:
def clean_dataframe(df, columns_to_drop=None, rename_columns=None):
    """
    Nettoie un DataFrame en supprimant les colonnes inutiles
    et en renommant les colonnes.
    """
    if columns_to_drop:
        df = df.drop(columns=columns_to_drop, errors="ignore")
    if rename_columns:
        df = df.rename(columns=rename_columns)
    # df.columns = [
    # unidecode(col.strip().lower()) for col in df.columns
    # ]  # Standardiser les noms de colonnes
    return df


# Exemple d'utilisation sur un DataFrame
# df_cleaned = clean_dataframe(dataframes[0], columns_to_drop=['Unnamed: 15'], rename_columns={'Country Name': 'Zone'})
# print(df_cleaned.head())

In [191]:
def handle_missing_values(df, strategy="mean"):
    """
    Gère les valeurs manquantes dans un DataFrame.
    - strategy='mean': Remplace par la moyenne (pour colonnes numériques)
    - strategy='mode': Remplace par le mode (pour colonnes catégoriques)
    - strategy='drop': Supprime les lignes avec des NaN
    """
    if strategy == "mean":
        for col in df.select_dtypes(include=[np.number]):
            df[col] = df[col].fillna(df[col].mean())
    elif strategy == "mode":
        for col in df.select_dtypes(include=[object]):
            df[col] = df[col].fillna(df[col].mode()[0])
    elif strategy == "drop":
        df = df.dropna()
    return df


# Exemple d'application
# df_cleaned = handle_missing_values(df_cleaned, strategy="mean")
# print(df_cleaned.isnull().sum())

In [192]:
# Fonction pour standardiser les noms (enlever accents, passer en minuscules, supprimer espaces inutiles)


def standardize_name(name):
    # Convertir en minuscules, enlever accents et nettoyer espaces
    return " ".join(unidecode(name.strip().lower()).split())

In [193]:
def inner_join(df1, df2, on, indicator=True):
    """
    Effectue une jointure interne entre deux DataFrames et fournit un rapport de qualité.

    Args:
        df1 (pandas.DataFrame): Premier DataFrame.
        df2 (pandas.DataFrame): Second DataFrame.
        on (str ou list): Colonne(s) sur laquelle(lesquelles) effectuer la jointure.
        indicator (bool, optionnel): Si True, ajoute une colonne '_merge' pour indiquer le type de jointure. Par défaut à True.

    Returns:
        pandas.DataFrame: Le DataFrame résultant de la jointure.
    """

    merged_df = pd.merge(df1, df2, on=on, how="inner", indicator=indicator)

    # Rapport de qualité de la jointure
    print("Rapport de qualité de la jointure :")
    quality_report = merged_df["_merge"].value_counts().reset_index()
    quality_report.columns = ["Type de jointure", "Nombre"]

    print(tabulate(quality_report, headers="keys", tablefmt="github"))

    return merged_df

In [194]:
# Fonction pour correspondance
def trouver_correspondances_pays(liste_pays_1, liste_pays_2, seuil=95):
    correspondances = []
    for pays in liste_pays_1:
        match = process.extractOne(pays, liste_pays_2)  # Trouve le meilleur match
        if match[1] >= seuil:  # Si le score dépasse le seuil
            correspondances.append(
                (pays, match[0], match[1])
            )  # Ajoute le pays et le score
        else:
            correspondances.append(
                (pays, None, match[1])
            )  # Pas de match au-dessus du seuil
    return pd.DataFrame(correspondances, columns=["Pays_Source", "Pays_Match", "Score"])

In [195]:
def rename_multiple_names(
    dataframes_dict, target_dataframes, col_name, correspondance_dict
):
    """
    Renomme plusieurs valeurs dans une colonne donnée pour des DataFrames spécifiques.

    Args:
        dataframes_dict (dict): Dictionnaire contenant des DataFrames avec leurs noms comme clés.
        target_dataframes (list): Liste des noms des DataFrames à modifier.
        col_name (str): Le nom de la colonne à modifier.
        correspondance_dict (dict): Dictionnaire de correspondances {"ancien_nom": "nouveau_nom"}.

    Returns:
        None. Les DataFrames sont modifiés en place.
    """
    for df_name in target_dataframes:
        if df_name in dataframes_dict:
            df = dataframes_dict[df_name]
            if col_name in df.columns:
                original_values = df[col_name].unique()

                # Remplacer les noms selon le dictionnaire de correspondance
                df[col_name] = df[col_name].replace(correspondance_dict)

                # Logger les changements effectués
                modified_values = df[col_name].unique()
                changes = [
                    (old, new)
                    for old, new in correspondance_dict.items()
                    if old in original_values and new in modified_values
                ]

                if changes:
                    print(f"Modifications dans le DataFrame '{df_name}':")
                    for old, new in changes:
                        print(f"  - '{old}' renommé en '{new}'")
            else:
                print(
                    f"La colonne '{col_name}' n'existe pas dans le DataFrame '{df_name}'"
                )
        else:
            print(
                f"Le DataFrame '{df_name}' n'est pas présent dans le dictionnaire fourni."
            )

<div style="background-color: #000000;" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 2 - Préparer et nettoyer les données</h2>
</div>

In [196]:
# Renommer la colonne 'Country Name' en 'Zone' dans chaque DataFrame
for name, df in dataframes_dict.items():
    if "Country Name" in df.columns:  # Vérifie si la colonne existe
        df.rename(columns={"Country Name": "Zone"}, inplace=True)
        print(f"Colonne 'Country Name' renommée en 'Zone' dans le DataFrame '{name}'.")

Colonne 'Country Name' renommée en 'Zone' dans le DataFrame 'couts_procedure_exportation_formalite_douaniere_usd'.
Colonne 'Country Name' renommée en 'Zone' dans le DataFrame 'terre_agricole_territoire_%'.


In [197]:
# Vérification
print("\n--- Aperçu des colonnes après renommage ---")
for name, df in dataframes_dict.items():
    print(f"{name}: {df.columns}")


--- Aperçu des colonnes après renommage ---
correspondance_pays-DESKTOP-7JGGVM9: Index(['Nom_Original', 'Nom_Correspondant'], dtype='object')
correspondance_pays: Index(['Zone,Nom_Correspondant'], dtype='object')
correspondance_pays_2: Index(['Nom_Original', 'Nom_Correspondant'], dtype='object')
couts_procedure_exportation_formalite_douaniere_usd: Index(['Zone', 'Country Code', 'Indicator Name', 'Indicator Code', '1960',
       '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969',
       '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978',
       '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'],
      dtype='object'

In [198]:
# Standardiser les noms dans la colonne 'Zone' de tous les DataFrames
for name, df in dataframes_dict.items():
    if "Zone" in df.columns:
        df["Zone"] = df["Zone"].astype(str).apply(standardize_name)
        print(f"Noms de la colonne 'Zone' standardisés dans le DataFrame '{name}'.")

Noms de la colonne 'Zone' standardisés dans le DataFrame 'couts_procedure_exportation_formalite_douaniere_usd'.
Noms de la colonne 'Zone' standardisés dans le DataFrame 'df_poule_qui_chante'.
Noms de la colonne 'Zone' standardisés dans le DataFrame 'DisponibiliteAlimentaire_2017'.
Noms de la colonne 'Zone' standardisés dans le DataFrame 'pib_par_habitant_ppa'.
Noms de la colonne 'Zone' standardisés dans le DataFrame 'Population_2000_2018'.
Noms de la colonne 'Zone' standardisés dans le DataFrame 'stabilite_politique_2017'.
Noms de la colonne 'Zone' standardisés dans le DataFrame 'taux_de_change_standardise'.
Noms de la colonne 'Zone' standardisés dans le DataFrame 'terre_agricole_territoire_%'.


In [199]:
# Collecter tous les noms uniques de pays
all_countries = set()
for name, df in dataframes_dict.items():
    if "Zone" in df.columns:
        countries = df["Zone"].dropna().astype(str).apply(standardize_name)
        all_countries.update(countries.unique())

In [200]:
# Créer une liste de correspondance avec noms standardisés
correspondance_pays = {country: country for country in sorted(all_countries)}

# Exporter la correspondance pour modification manuelle
df_correspondance = pd.DataFrame(
    list(correspondance_pays.items()), columns=["Zone", "Nom_Correspondant"]
)
df_correspondance.to_csv("correspondance_pays.csv", index=False, encoding="utf-8")

print("Table de correspondance générée et sauvegardée dans 'correspondance_pays.csv'.")

Table de correspondance générée et sauvegardée dans 'correspondance_pays.csv'.


In [201]:
# Charger la table de correspondance modifiée
df_correspondance = pd.read_csv("correspondance_pays.csv", sep=",")
correspondance_dict = dict(
    zip(df_correspondance["Zone"], df_correspondance["Nom_Correspondant"])
)

In [202]:
# Afficher la table de correspondance
df_correspondance.head()

,Zone,Nom_Correspondant
0,afghanistan,afghanistan
1,afrique du nord et moyen-orient,afrique du nord et moyen-orient
2,afrique du nord et moyen-orient (bird et ida),afrique du nord et moyen-orient (bird et ida)
3,afrique du nord et moyen-orient (hors revenu e...,afrique du nord et moyen-orient (hors revenu e...
4,afrique du sud,afrique du sud


<h3 style="margin: auto; padding: 20px; color: #000000; ">2.1 - Analyse du fichier population </h3>
</div>

In [203]:
# Nom du DataFrame sélectionné
df_name = "Population_2000_2018"

# Accéder au DataFrame correspondant dans le dictionnaire
df_population = dataframes_dict.get(df_name)

# Afficher le dataframe
df_population.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.953,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.988,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,22600.770,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,23680.871,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.684,X,Sources internationales sûres,NaN


In [204]:
# Afficher le descriptif des variables
description_variables(df_population)

Description donnée(s):
Le jeu de données comporte: 4411 observation(s)
Le jeu de données comporte: 15 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Note,object,4411,258,1,4153,94.15
Code Domaine,object,4411,4411,1,0,0.00
Domaine,object,4411,4411,1,0,0.00
Code zone,int64,4411,4411,238,0,0.00
Zone,object,4411,4411,238,0,0.00
Code Élément,int64,4411,4411,1,0,0.00
Élément,object,4411,4411,1,0,0.00
Code Produit,int64,4411,4411,1,0,0.00
Produit,object,4411,4411,1,0,0.00
Code année,int64,4411,4411,19,0,0.00


In [205]:
# Filtrer les données de l'année 2017
df_population_2017 = df_population.loc[df_population["Année"] == 2017, :]
df_population_2017.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
17,OA,Séries temporelles annuelles,2,afghanistan,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,36296.113,X,Sources internationales sûres,NaN
36,OA,Séries temporelles annuelles,202,afrique du sud,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,57009.756,X,Sources internationales sûres,NaN
55,OA,Séries temporelles annuelles,3,albanie,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,2884.169,X,Sources internationales sûres,NaN
74,OA,Séries temporelles annuelles,4,algerie,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,41389.189,X,Sources internationales sûres,NaN
93,OA,Séries temporelles annuelles,79,allemagne,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,82658.409,X,Sources internationales sûres,NaN


In [206]:
# Supprimer les colonnes inutiles
clean_dataframe(
    df_population_2017,
    columns_to_drop=[
        "Code Domaine",
        "Domaine",
        "Description du Symbole",
        "Symbole",
        "Note",
    ],
    rename_columns=None,
)

# Afficher le dataframe
df_population_2017.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
17,OA,Séries temporelles annuelles,2,afghanistan,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,36296.113,X,Sources internationales sûres,NaN
36,OA,Séries temporelles annuelles,202,afrique du sud,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,57009.756,X,Sources internationales sûres,NaN
55,OA,Séries temporelles annuelles,3,albanie,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,2884.169,X,Sources internationales sûres,NaN
74,OA,Séries temporelles annuelles,4,algerie,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,41389.189,X,Sources internationales sûres,NaN
93,OA,Séries temporelles annuelles,79,allemagne,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,82658.409,X,Sources internationales sûres,NaN


In [207]:
# Afficher les descriptif des variables
description_variables(df_population_2017)

Description donnée(s):
Le jeu de données comporte: 236 observation(s)
Le jeu de données comporte: 15 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Note,object,236,1,1,235,99.58
Code Domaine,object,236,236,1,0,0.00
Domaine,object,236,236,1,0,0.00
Code zone,int64,236,236,236,0,0.00
Zone,object,236,236,236,0,0.00
Code Élément,int64,236,236,1,0,0.00
Élément,object,236,236,1,0,0.00
Code Produit,int64,236,236,1,0,0.00
Produit,object,236,236,1,0,0.00
Code année,int64,236,236,1,0,0.00


In [208]:
# Convertir la colonne Année au format date
df_population_2017["Année"] = pd.to_datetime(df_population_2017["Année"], format="%Y")

C:\Users\jerem\AppData\Local\Temp\ipykernel_15568\1382938700.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [209]:
# Vérifier les modifications
description_variables(df_population_2017)

Description donnée(s):
Le jeu de données comporte: 236 observation(s)
Le jeu de données comporte: 15 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Note,object,236,1,1,235,99.58
Code Domaine,object,236,236,1,0,0.00
Domaine,object,236,236,1,0,0.00
Code zone,int64,236,236,236,0,0.00
Zone,object,236,236,236,0,0.00
Code Élément,int64,236,236,1,0,0.00
Élément,object,236,236,1,0,0.00
Code Produit,int64,236,236,1,0,0.00
Produit,object,236,236,1,0,0.00
Code année,int64,236,236,1,0,0.00


In [210]:
# Harmoniser les unités.Multiplication de la colonne valeur par 1000
df_population_2017["population (millions hab)"] = df_population_2017["Valeur"] * 1000

C:\Users\jerem\AppData\Local\Temp\ipykernel_15568\3508033930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [211]:
# Classer la colonne Valeur par ordre croissant
df_population_2017 = df_population_2017.sort_values(by="Valeur", ascending=True)
df_population_2017.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note,population (millions hab)
3672,OA,Séries temporelles annuelles,94,saint-siege,511,Population totale,3010,Population-Estimations,2017,2017-01-01,1000 personnes,0.793,X,Sources internationales sûres,NaN,793.0
4143,OA,Séries temporelles annuelles,218,tokelaou,511,Population totale,3010,Population-Estimations,2017,2017-01-01,1000 personnes,1.300,X,Sources internationales sûres,"UNDESA, Population Division – World Population...",1300.0
2896,OA,Séries temporelles annuelles,160,nioue,511,Population totale,3010,Population-Estimations,2017,2017-01-01,1000 personnes,1.609,X,Sources internationales sûres,NaN,1609.0
1781,OA,Séries temporelles annuelles,65,iles falkland (malvinas),511,Population totale,3010,Population-Estimations,2017,2017-01-01,1000 personnes,3.068,X,Sources internationales sûres,NaN,3068.0
2725,OA,Séries temporelles annuelles,142,montserrat,511,Population totale,3010,Population-Estimations,2017,2017-01-01,1000 personnes,4.984,X,Sources internationales sûres,NaN,4984.0


In [212]:
# Conserver les colonnes utiles
df_population_2017 = df_population_2017[
    ["Code zone", "Zone", "population (millions hab)"]
]

# Afficher le dataframe
df_population_2017.head()

,Code zone,Zone,population (millions hab)
3672,94,saint-siege,793.0
4143,218,tokelaou,1300.0
2896,160,nioue,1609.0
1781,65,iles falkland (malvinas),3068.0
2725,142,montserrat,4984.0


In [213]:
# Description du dataframe
description_variables(df_population_2017)

Description donnée(s):
Le jeu de données comporte: 236 observation(s)
Le jeu de données comporte: 3 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Code zone,int64,236,236,236,0,0.0
Zone,object,236,236,236,0,0.0
population (millions hab),float64,236,236,236,0,0.0


In [214]:
# Calculer la population totale
population_totale_2017 = df_population_2017["population (millions hab)"].sum()
population_totale_2017

7548134111.0

Le dataset ne comporte pas de données manquantes, ni doublons.

<h3 style="margin: auto; padding: 20px; color: #000000; ">2.2 - Analyse du fichier Disponibilité Alimentaire pour l'année 2017 </h3>
</div>

In [215]:
# Nom du DataFrame sélectionné
df_name = "DisponibiliteAlimentaire_2017"

# Accéder au DataFrame correspondant dans le dictionnaire
df_dispo_alimentaire_2017 = dataframes_dict.get(df_name)

# Afficher le dataframe
df_dispo_alimentaire_2017.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,"4281,00",S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,"2302,00",S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,"-119,00",S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,"0,00",S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,"6701,00",S,Données standardisées


In [216]:
# Afficher le descriptif des variables
description_variables(df_dispo_alimentaire_2017)

Description donnée(s):
Le jeu de données comporte: 176600 observation(s)
Le jeu de données comporte: 14 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Code Domaine,object,176600,176600,1,0,0.0
Domaine,object,176600,176600,1,0,0.0
Code zone,int64,176600,176600,174,0,0.0
Zone,object,176600,176600,174,0,0.0
Code Élément,int64,176600,176600,17,0,0.0
Élément,object,176600,176600,17,0,0.0
Code Produit,int64,176600,176600,98,0,0.0
Produit,object,176600,176600,98,0,0.0
Code année,int64,176600,176600,1,0,0.0
Année,int64,176600,176600,1,0,0.0


In [217]:
# Identifier la catégorie volaille dans la colonne produit
valeurs_produit = df_dispo_alimentaire_2017["Produit"].unique()
print(valeurs_produit)

['Blé et produits' 'Riz et produits' 'Orge et produits' 'Maïs et produits'
 'Seigle et produits' 'Avoine' 'Millet et produits' 'Sorgho et produits'
 'Céréales, Autres' 'Pommes de Terre et produits' 'Ignames' 'Racines nda'
 'Sucre, canne' 'Sucre, betterave' 'Sucre Eq Brut' 'Edulcorants Autres'
 'Miel' 'Haricots' 'Pois' 'Légumineuses Autres et produits'
 'Noix et produits' 'Soja' 'Arachides Decortiquees' 'Graines de tournesol'
 'Graines Colza/Moutarde' 'Graines de coton' 'Coco (Incl Coprah)' 'Sésame'
 'Olives' 'Plantes Oleiferes, Autre' 'Huile de Soja' "Huile d'Arachide"
 'Huile de Tournesol' 'Huile de Colza&Moutarde' 'Huile Graines de Coton'
 'Huile de Palmistes' 'Huile de Palme' 'Huile de Coco' 'Huile de Sésame'
 "Huile d'Olive" 'Huile de Son de Riz' 'Huile de Germe de Maïs'
 'Huil Plantes Oleif Autr' 'Tomates et produits' 'Oignons'
 'Légumes, Autres' 'Oranges, Mandarines' 'Citrons & Limes et produits'
 'Pamplemousse et produits' 'Agrumes, Autres' 'Bananes'
 'Pommes et produits' 'Anana

In [218]:
# Filter le dataframe en fonction de la catégorie de produit Viande de Volailles
categorie_produit = ["Viande de Volailles"]
df_dispo_alimentaire_2017 = df_filtre = df_dispo_alimentaire_2017.loc[
    df_dispo_alimentaire_2017["Produit"].isin(categorie_produit)
]

# Afficher le dataframe
df_dispo_alimentaire_2017.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
651,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5511,Production,2734,Viande de Volailles,2017,2017,Milliers de tonnes,"28,00",S,Données standardisées
652,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5611,Importations - Quantité,2734,Viande de Volailles,2017,2017,Milliers de tonnes,"29,00",S,Données standardisées
653,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5072,Variation de stock,2734,Viande de Volailles,2017,2017,Milliers de tonnes,"0,00",S,Données standardisées
654,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5301,Disponibilité intérieure,2734,Viande de Volailles,2017,2017,Milliers de tonnes,"57,00",S,Données standardisées
655,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5123,Pertes,2734,Viande de Volailles,2017,2017,Milliers de tonnes,"2,00",S,Données standardisées


In [219]:
# Supprimer les colonnes inutiles
clean_dataframe(
    df_population_2017,
    columns_to_drop=[
        "Code Domaine",
        "Domaine",
        "Description du Symbole",
        "Symbole",
        "Unité",
        "Code Élément",
        "Code Produit",
    ],
    rename_columns=None,
)

# Afficher le dataframe
df_dispo_alimentaire_2017.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
651,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5511,Production,2734,Viande de Volailles,2017,2017,Milliers de tonnes,"28,00",S,Données standardisées
652,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5611,Importations - Quantité,2734,Viande de Volailles,2017,2017,Milliers de tonnes,"29,00",S,Données standardisées
653,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5072,Variation de stock,2734,Viande de Volailles,2017,2017,Milliers de tonnes,"0,00",S,Données standardisées
654,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5301,Disponibilité intérieure,2734,Viande de Volailles,2017,2017,Milliers de tonnes,"57,00",S,Données standardisées
655,FBS,Nouveaux Bilans Alimentaire,2,afghanistan,5123,Pertes,2734,Viande de Volailles,2017,2017,Milliers de tonnes,"2,00",S,Données standardisées


In [220]:
# Pivoter la colonne Elément du dataframe
df_dispo_alimentaire_2017_pivot = df_dispo_alimentaire_2017.pivot_table(
    index=["Code zone", "Zone"], columns="Élément", values="Valeur", aggfunc="sum"
)

In [221]:
# Afficher le df pivoté
df_dispo_alimentaire_2017_pivot.head()

,Élément,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
Code zone,Zone,,,,,,,,,,,,,,,,,
1,armenie,NaN,NaN,NaN,"54,00","16,06","3,39","5,44","47,00","0,00","35,00","47,00","0,00","11,00","0,00",NaN,NaN,"-2,00"
2,afghanistan,NaN,NaN,NaN,"5,00","1,53","0,33","0,54","57,00",NaN,"29,00","55,00","2,00","28,00","0,00",NaN,NaN,"0,00"
3,albanie,NaN,NaN,NaN,"85,00","16,36","6,45","6,26","47,00","0,00","38,00","47,00",NaN,"13,00","0,00",NaN,NaN,"4,00"
4,algerie,"0,00",NaN,NaN,"22,00","6,38","1,50","1,97","277,00","0,00","2,00","264,00","13,00","275,00","0,00",NaN,NaN,"0,00"
7,angola,"0,00",NaN,NaN,"35,00","10,56","2,22","3,60","319,00","0,00","277,00","315,00","2,00","42,00","0,00",NaN,"2,00","0,00"


In [222]:
# Description des variables du df pivoté
description_variables(df_dispo_alimentaire_2017_pivot)

Description donnée(s):
Le jeu de données comporte: 172 observation(s)
Le jeu de données comporte: 17 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Élément,,,,,,
Aliments pour animaux,object,172,1,1,171,99.42
Semences,object,172,1,1,171,99.42
Autres utilisations (non alimentaire),object,172,34,18,138,80.23
Traitement,object,172,46,25,126,73.26
Pertes,object,172,67,32,105,61.05
Alimentation pour touristes,object,172,78,7,94,54.65
Exportations - Quantité,object,172,135,50,37,21.51
Résidus,object,172,164,15,8,4.65
Production,object,172,168,125,4,2.33


In [223]:
# Conserver les colonnes utiles
df_dispo_alimentaire_2017 = df_dispo_alimentaire_2017_pivot[
    [
        "Disponibilité alimentaire en quantité (kg/personne/an)",
        "Importations - Quantité",
        "Production",
        "Exportations - Quantité",
        "Disponibilité intérieure",
    ]
]
df_dispo_alimentaire_2017 = df_dispo_alimentaire_2017.reset_index()

# Afficher le dataframe
df_dispo_alimentaire_2017.head()

Élément,Code zone,Zone,Disponibilité alimentaire en quantité (kg/personne/an),Importations - Quantité,Production,Exportations - Quantité,Disponibilité intérieure
0,1,armenie,"16,06","35,00","11,00","0,00","47,00"
1,2,afghanistan,"1,53","29,00","28,00",NaN,"57,00"
2,3,albanie,"16,36","38,00","13,00","0,00","47,00"
3,4,algerie,"6,38","2,00","275,00","0,00","277,00"
4,7,angola,"10,56","277,00","42,00","0,00","319,00"


In [224]:
# Description du dataframe
description_variables(df_dispo_alimentaire_2017)

Description donnée(s):
Le jeu de données comporte: 172 observation(s)
Le jeu de données comporte: 7 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Élément,,,,,,
Exportations - Quantité,object,172,135,50,37,21.51
Production,object,172,168,125,4,2.33
Importations - Quantité,object,172,170,84,2,1.16
Disponibilité intérieure,object,172,170,135,2,1.16
Code zone,int64,172,172,172,0,0.00
Zone,object,172,172,172,0,0.00
Disponibilité alimentaire en quantité (kg/personne/an),object,172,172,169,0,0.00


In [225]:
colonnes_a_convertir = [
    "Exportations - Quantité",
    "Production",
    "Importations - Quantité",
    "Disponibilité intérieure",
    "Disponibilité alimentaire en quantité (kg/personne/an)",
]

# Supprimer les caractères parasites avant conversion
for col in colonnes_a_convertir:
    df_dispo_alimentaire_2017[col] = (
        df_dispo_alimentaire_2017[col]
        .astype(str)  # Convertir en chaîne
        .str.replace(
            r"[^\d.,-]", "", regex=True
        )  # Garder uniquement les chiffres, points et virgules
        .str.replace(",", ".")  # Remplacer les virgules par des points
    )
    df_dispo_alimentaire_2017[col] = pd.to_numeric(
        df_dispo_alimentaire_2017[col], errors="coerce"
    )  # Convertir en numérique

# Description du dataframe
description_variables(df_dispo_alimentaire_2017)

Description donnée(s):
Le jeu de données comporte: 172 observation(s)
Le jeu de données comporte: 7 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Élément,,,,,,
Exportations - Quantité,float64,172,135,50,37,21.51
Production,float64,172,168,125,4,2.33
Importations - Quantité,float64,172,170,84,2,1.16
Disponibilité intérieure,float64,172,170,135,2,1.16
Code zone,int64,172,172,172,0,0.00
Zone,object,172,172,172,0,0.00
Disponibilité alimentaire en quantité (kg/personne/an),float64,172,172,169,0,0.00


<h3 style="margin: auto; padding: 20px; color: #000000; ">2.3 - Analyse du fichier stabilité politique </h3>
</div>

In [226]:
# Nom du DataFrame sélectionné
df_name = "stabilite_politique_2017"

# Accéder au DataFrame correspondant dans le dictionnaire
df_stabilite_politique = dataframes_dict.get(df_name)

# Afficher le dataframe
df_stabilite_politique.head()

,Zone,Année,Stabilite Politique
0,afghanistan,2017,"-2,8"
1,afrique du sud,2017,"-0,28"
2,albanie,2017,"0,38"
3,algerie,2017,"-0,92"
4,allemagne,2017,"0,59"


In [227]:
# Afficher la description des variables
description_variables(df_stabilite_politique)

Description donnée(s):
Le jeu de données comporte: 198 observation(s)
Le jeu de données comporte: 3 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Zone,object,198,198,198,0,0.0
Année,int64,198,198,1,0,0.0
Stabilite Politique,object,198,198,152,0,0.0


In [228]:
# Conserver les colonnes utiles
df_stabilite_politique = df_stabilite_politique[["Zone", "Stabilite Politique"]]


# Afficher le dataframe
df_stabilite_politique.head()

,Zone,Stabilite Politique
0,afghanistan,"-2,8"
1,afrique du sud,"-0,28"
2,albanie,"0,38"
3,algerie,"-0,92"
4,allemagne,"0,59"


In [229]:
colonnes_a_convertir = ["Stabilite Politique"]

# Supprimer les caractères parasites avant conversion
for col in colonnes_a_convertir:
    df_stabilite_politique[col] = (
        df_stabilite_politique[col]
        .astype(str)  # Convertir en chaîne
        .str.replace(
            r"[^\d.,-]", "", regex=True
        )  # Garder uniquement les chiffres, points et virgules
        .str.replace(",", ".")  # Remplacer les virgules par des points
    )
    df_stabilite_politique[col] = pd.to_numeric(
        df_stabilite_politique[col], errors="coerce"
    )  # Convertir en numérique

C:\Users\jerem\AppData\Local\Temp\ipykernel_15568\1036815024.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jerem\AppData\Local\Temp\ipykernel_15568\1036815024.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [230]:
# Description du dataframe
description_variables(df_stabilite_politique)

Description donnée(s):
Le jeu de données comporte: 198 observation(s)
Le jeu de données comporte: 2 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Zone,object,198,198,198,0,0.0
Stabilite Politique,float64,198,198,152,0,0.0


Le dataset ne comporte pas de données manquantes, ni doublons.

<h3 style="margin: auto; padding: 20px; color: #000000; ">2.4 - Analyse du fichier - Taux de change standardise</h3>
</div>

In [231]:
# Nom du DataFrame sélectionné
df_name = "taux_de_change_standardise"

# Accéder au DataFrame correspondant dans le dictionnaire
df_taux_change = dataframes_dict.get(df_name)

# Afficher le dataframe
df_taux_change.head()

,Code Domaine,Domaine,Code zone (M49),Zone,Code de devise ISO (FAO),Devise,Code Élément,Élément,Code année,Année,Code Mois,Mois,Unité,Valeur,Symbole,Description du Symbole
0,PE,Taux de change,4,afghanistan,AFN,Afghani,SLC,Unités de monnaie locale standardisée par USD,2017,2017,7021,Valeur annuel,NaN,"68,02690408",X,Ciffre de sources internationales
1,PE,Taux de change,710,afrique du sud,ZAR,Rand,SLC,Unités de monnaie locale standardisée par USD,2017,2017,7021,Valeur annuel,NaN,"13,33378146",X,Ciffre de sources internationales
2,PE,Taux de change,8,albanie,ALL,Lek,SLC,Unités de monnaie locale standardisée par USD,2017,2017,7021,Valeur annuel,NaN,"119,0996667",X,Ciffre de sources internationales
3,PE,Taux de change,12,algerie,DZD,Algerian Dinar,SLC,Unités de monnaie locale standardisée par USD,2017,2017,7021,Valeur annuel,NaN,"110,9730167",X,Ciffre de sources internationales
4,PE,Taux de change,276,allemagne,EUR,Euro,SLC,Unités de monnaie locale standardisée par USD,2017,2017,7021,Valeur annuel,NaN,"0,885205508",X,Ciffre de sources internationales


In [232]:
# Afficher le descriptif des variables
df_taux_change.head()

,Code Domaine,Domaine,Code zone (M49),Zone,Code de devise ISO (FAO),Devise,Code Élément,Élément,Code année,Année,Code Mois,Mois,Unité,Valeur,Symbole,Description du Symbole
0,PE,Taux de change,4,afghanistan,AFN,Afghani,SLC,Unités de monnaie locale standardisée par USD,2017,2017,7021,Valeur annuel,NaN,"68,02690408",X,Ciffre de sources internationales
1,PE,Taux de change,710,afrique du sud,ZAR,Rand,SLC,Unités de monnaie locale standardisée par USD,2017,2017,7021,Valeur annuel,NaN,"13,33378146",X,Ciffre de sources internationales
2,PE,Taux de change,8,albanie,ALL,Lek,SLC,Unités de monnaie locale standardisée par USD,2017,2017,7021,Valeur annuel,NaN,"119,0996667",X,Ciffre de sources internationales
3,PE,Taux de change,12,algerie,DZD,Algerian Dinar,SLC,Unités de monnaie locale standardisée par USD,2017,2017,7021,Valeur annuel,NaN,"110,9730167",X,Ciffre de sources internationales
4,PE,Taux de change,276,allemagne,EUR,Euro,SLC,Unités de monnaie locale standardisée par USD,2017,2017,7021,Valeur annuel,NaN,"0,885205508",X,Ciffre de sources internationales


In [233]:
# Liste des colonnes à conserver
colonnes_a_conserver = ["Code zone (M49)", "Zone", "Valeur"]

# Filtrage du DataFrame
df_taux_change = df_taux_change.loc[:, colonnes_a_conserver]

# Renommer la colonne Valeur
df_taux_change.rename(columns={"Valeur": "Taux de change"}, inplace=True)

# Affichage des premières lignes pour vérifier
df_taux_change.head()

,Code zone (M49),Zone,Taux de change
0,4,afghanistan,"68,02690408"
1,710,afrique du sud,"13,33378146"
2,8,albanie,"119,0996667"
3,12,algerie,"110,9730167"
4,276,allemagne,"0,885205508"


In [234]:
# Description du dataframe
description_variables(df_taux_change)

Description donnée(s):
Le jeu de données comporte: 210 observation(s)
Le jeu de données comporte: 3 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Code zone (M49),int64,210,210,210,0,0.0
Zone,object,210,210,210,0,0.0
Taux de change,object,210,210,140,0,0.0


In [235]:
colonnes_a_convertir = ["Taux de change"]

# Supprimer les caractères parasites avant conversion
for col in colonnes_a_convertir:
    df_taux_change[col] = (
        df_taux_change[col]
        .astype(str)  # Convertir en chaîne
        .str.replace(
            r"[^\d.,-]", "", regex=True
        )  # Garder uniquement les chiffres, points et virgules
        .str.replace(",", ".")  # Remplacer les virgules par des points
    )
    df_taux_change[col] = pd.to_numeric(
        df_taux_change[col], errors="coerce"
    )  # Convertir en numérique

# description du dataframe
description_variables(df_taux_change)

Description donnée(s):
Le jeu de données comporte: 210 observation(s)
Le jeu de données comporte: 3 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Taux de change,float64,210,209,139,1,0.48
Code zone (M49),int64,210,210,210,0,0.00
Zone,object,210,210,210,0,0.00


<h3 style="margin: auto; padding: 20px; color: #000000; ">2.5 - Analyse du fichier - Coûts des procédures d'exportation, formalité douanières en USD</h3>
</div>

In [236]:
# Nom du DataFrame sélectionné
df_name = "couts_procedure_exportation_formalite_douaniere_usd"

# Accéder au DataFrame correspondant dans le dictionnaire
df_couts_procedure_exportation = dataframes_dict.get(df_name)

# Afficher le dataframe
df_couts_procedure_exportation.head()

,Zone,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,aruba,ABW,"Coût des procédures d’exportation, formalités ...",IC.EXP.CSBC.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,nan,AFE,"Coût des procédures d’exportation, formalités ...",IC.EXP.CSBC.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"553,52","557,78","557,78","557,78","557,78","557,78",NaN,NaN,NaN,NaN
2,afghanistan,AFG,"Coût des procédures d’exportation, formalités ...",IC.EXP.CSBC.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"511,11","511,11","452,78","452,78","452,78","452,78",NaN,NaN,NaN,NaN
3,nan,AFW,"Coût des procédures d’exportation, formalités ...",IC.EXP.CSBC.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"653,92","653,92","660,99","654,67","654,67","654,67",NaN,NaN,NaN,NaN
4,angola,AGO,"Coût des procédures d’exportation, formalités ...",IC.EXP.CSBC.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,825,825,825,825,825,825,NaN,NaN,NaN,NaN


In [237]:
# Description du dataframe
description_variables(df_couts_procedure_exportation)

Description donnée(s):
Le jeu de données comporte: 267 observation(s)
Le jeu de données comporte: 68 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
1990,float64,267,0,0,267,100.00
1986,float64,267,0,0,267,100.00
1988,float64,267,0,0,267,100.00
1989,float64,267,0,0,267,100.00
1991,float64,267,0,0,267,100.00
1992,float64,267,0,0,267,100.00
1993,float64,267,0,0,267,100.00
1994,float64,267,0,0,267,100.00
1995,float64,267,0,0,267,100.00
1996,float64,267,0,0,267,100.00


In [238]:
# Liste des colonnes à conserver
colonnes_a_conserver = ["Zone", "Country Code", "Indicator Name", "2017"]

# Filtrage du DataFrame
df_couts_procedure_exportation = df_couts_procedure_exportation.loc[
    :, colonnes_a_conserver
]

# Affichage des premières lignes pour vérifier
df_couts_procedure_exportation.head()

,Zone,Country Code,Indicator Name,2017
0,aruba,ABW,"Coût des procédures d’exportation, formalités ...",NaN
1,nan,AFE,"Coût des procédures d’exportation, formalités ...","557,78"
2,afghanistan,AFG,"Coût des procédures d’exportation, formalités ...","452,78"
3,nan,AFW,"Coût des procédures d’exportation, formalités ...","654,67"
4,angola,AGO,"Coût des procédures d’exportation, formalités ...",825


In [239]:
# Renommer les colonnes Country Name et 2017
df_couts_procedure_exportation = clean_dataframe(
    df_couts_procedure_exportation,
    columns_to_drop=None,
    rename_columns={"Country Name": "Zone", "2017": "Coûts Procedures d'Exportation"},
)

# Afficher le dataframe
df_couts_procedure_exportation.head()

,Zone,Country Code,Indicator Name,Coûts Procedures d'Exportation
0,aruba,ABW,"Coût des procédures d’exportation, formalités ...",NaN
1,nan,AFE,"Coût des procédures d’exportation, formalités ...","557,78"
2,afghanistan,AFG,"Coût des procédures d’exportation, formalités ...","452,78"
3,nan,AFW,"Coût des procédures d’exportation, formalités ...","654,67"
4,angola,AGO,"Coût des procédures d’exportation, formalités ...",825


In [240]:
# Retourner uniquement le nom des pays
# Afficher les lignes qui contiennet des valeurs NaN
df_couts_procedure_exportation_nan = df_couts_procedure_exportation[
    df_couts_procedure_exportation.isna().any(axis=1)
]
country_names_nan = df_couts_procedure_exportation_nan["Zone"]
country_names_nan.values

array(['aruba', 'andorre', 'samoa americaines', 'bermudes',
       'iles anglo-normandes', 'cuba', 'curacao', 'iles caimans',
       'erythree', 'iles feroe', 'gibraltar', 'groenland', 'guam',
       'ile de man', 'non classifie', 'liechtenstein',
       'region administrative speciale de macao', 'saint-martin (fr)',
       'monaco', 'mariannes', 'nouvelle-caledonie', 'nauru', 'coree',
       'polynesie francaise', 'sint maarten (dutch part)',
       'iles turques-et-caiques', 'turkmenistan', 'tuvalu',
       'iles vierges britanniques', 'iles vierges (eu)', 'yemen', 'nan'],
      dtype=object)

In [241]:
# Afficher les lignes qui contiennet des valeurs NaN
df_couts_procedure_exportation_nan = df_couts_procedure_exportation[
    df_couts_procedure_exportation.isna().any(axis=1)
]
df_couts_procedure_exportation_nan.head()

,Zone,Country Code,Indicator Name,Coûts Procedures d'Exportation
0,aruba,ABW,"Coût des procédures d’exportation, formalités ...",NaN
6,andorre,AND,"Coût des procédures d’exportation, formalités ...",NaN
11,samoa americaines,ASM,"Coût des procédures d’exportation, formalités ...",NaN
27,bermudes,BMU,"Coût des procédures d’exportation, formalités ...",NaN
38,iles anglo-normandes,CHI,"Coût des procédures d’exportation, formalités ...",NaN


In [243]:
colonnes_a_convertir = ["Coûts Procedures d'Exportation"]

# Supprimer les caractères parasites avant conversion
for col in colonnes_a_convertir:
    df_couts_procedure_exportation[col] = (
        df_couts_procedure_exportation[col]
        .astype(str)  # Convertir en chaîne
        .str.replace(
            r"[^\d.,-]", "", regex=True
        )  # Garder uniquement les chiffres, points et virgules
        .str.replace(",", ".")  # Remplacer les virgules par des points
    )
    df_couts_procedure_exportation[col] = pd.to_numeric(
        df_couts_procedure_exportation[col], errors="coerce"
    )  # Convertir en numérique

# description du dataframe
description_variables(df_couts_procedure_exportation)

Description donnée(s):
Le jeu de données comporte: 267 observation(s)
Le jeu de données comporte: 4 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Coûts Procedures d'Exportation,float64,267,235,198,32,11.99
Indicator Name,object,267,266,7,1,0.37
Zone,object,267,267,263,0,0.00
Country Code,object,267,267,267,0,0.00


In [244]:
# Description du dataframe
description_variables(df_couts_procedure_exportation)

Description donnée(s):
Le jeu de données comporte: 267 observation(s)
Le jeu de données comporte: 4 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Coûts Procedures d'Exportation,float64,267,235,198,32,11.99
Indicator Name,object,267,266,7,1,0.37
Zone,object,267,267,263,0,0.00
Country Code,object,267,267,267,0,0.00


In [245]:
# Convertir toutes les valeurs en chaînes, en remplaçant les NaN par une chaîne vide
df_couts_procedure_exportation["Zone"] = (
    df_couts_procedure_exportation["Zone"].fillna("").astype(str)
)

In [246]:
# Description du dataframe
description_variables(df_couts_procedure_exportation)

Description donnée(s):
Le jeu de données comporte: 267 observation(s)
Le jeu de données comporte: 4 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Coûts Procedures d'Exportation,float64,267,235,198,32,11.99
Indicator Name,object,267,266,7,1,0.37
Zone,object,267,267,263,0,0.00
Country Code,object,267,267,267,0,0.00


Il manque 32 observations dans ce jeu de données

<h3 style="margin: auto; padding: 20px; color: #000000; ">2.6 - Analyse du fichier - PIB par habitant - PPA</h3>
</div>

In [247]:
# Nom du DataFrame sélectionné
df_name = "pib_par_habitant_ppa"

# Accéder au DataFrame correspondant dans le dictionnaire
df_pib_par_habitant = dataframes_dict.get(df_name)

# Afficher le dataframe
df_pib_par_habitant.head()

,Code Domaine,Domaine,Code zone (M49),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,FS,Données de la sécurité alimentaire,4,afghanistan,6126,Valeur,22013,"PIB par habitant, ($ PPA internationaux consta...",2017,2017,Int$/cap,"2956,8",X,Ciffre de sources internationales,NaN
1,FS,Données de la sécurité alimentaire,710,afrique du sud,6126,Valeur,22013,"PIB par habitant, ($ PPA internationaux consta...",2017,2017,Int$/cap,"14823,6",X,Ciffre de sources internationales,NaN
2,FS,Données de la sécurité alimentaire,8,albanie,6126,Valeur,22013,"PIB par habitant, ($ PPA internationaux consta...",2017,2017,Int$/cap,"14229,3",X,Ciffre de sources internationales,NaN
3,FS,Données de la sécurité alimentaire,12,algerie,6126,Valeur,22013,"PIB par habitant, ($ PPA internationaux consta...",2017,2017,Int$/cap,"13805,4",X,Ciffre de sources internationales,NaN
4,FS,Données de la sécurité alimentaire,276,allemagne,6126,Valeur,22013,"PIB par habitant, ($ PPA internationaux consta...",2017,2017,Int$/cap,"61563,6",X,Ciffre de sources internationales,NaN


In [248]:
# Description du dataframe
description_variables(df_pib_par_habitant)

Description donnée(s):
Le jeu de données comporte: 190 observation(s)
Le jeu de données comporte: 15 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Note,float64,190,0,0,190,100.0
Code Domaine,object,190,190,1,0,0.0
Domaine,object,190,190,1,0,0.0
Code zone (M49),int64,190,190,190,0,0.0
Zone,object,190,190,190,0,0.0
Code Élément,int64,190,190,1,0,0.0
Élément,object,190,190,1,0,0.0
Code Produit,int64,190,190,1,0,0.0
Produit,object,190,190,1,0,0.0
Code année,int64,190,190,1,0,0.0


In [249]:
# Conserver les colonnes utiles
df_pib_par_habitant = df_pib_par_habitant[["Code zone (M49)", "Zone", "Valeur"]]

# Renommer la colonne Valeur
df_pib_par_habitant.rename(columns={"Valeur": "PIB par habitant - PPA"}, inplace=True)

# Afficher le dataframe
df_pib_par_habitant.head()

C:\Users\jerem\AppData\Local\Temp\ipykernel_15568\35298738.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Code zone (M49),Zone,PIB par habitant - PPA
0,4,afghanistan,"2956,8"
1,710,afrique du sud,"14823,6"
2,8,albanie,"14229,3"
3,12,algerie,"13805,4"
4,276,allemagne,"61563,6"


In [250]:
colonnes_a_convertir = ["PIB par habitant - PPA"]

# Supprimer les caractères parasites avant conversion
for col in colonnes_a_convertir:
    df_pib_par_habitant[col] = (
        df_pib_par_habitant[col]
        .astype(str)  # Convertir en chaîne
        .str.replace(
            r"[^\d.,-]", "", regex=True
        )  # Garder uniquement les chiffres, points et virgules
        .str.replace(",", ".")  # Remplacer les virgules par des points
    )
    df_pib_par_habitant[col] = pd.to_numeric(
        df_pib_par_habitant[col], errors="coerce"
    )  # Convertir en numérique

    # description du dataframe
    description_variables(df_pib_par_habitant)

Description donnée(s):
Le jeu de données comporte: 190 observation(s)
Le jeu de données comporte: 3 colonne(s)


C:\Users\jerem\AppData\Local\Temp\ipykernel_15568\300055259.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jerem\AppData\Local\Temp\ipykernel_15568\300055259.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [251]:
# Description du dataframe
description_variables(df_pib_par_habitant)

Description donnée(s):
Le jeu de données comporte: 190 observation(s)
Le jeu de données comporte: 3 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Code zone (M49),int64,190,190,190,0,0.0
Zone,object,190,190,190,0,0.0
PIB par habitant - PPA,float64,190,190,190,0,0.0


<div style="background-color: #000000;" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 3 - Jointures des différents fichiers</h2>
</div>

<h3 style="margin: auto; padding: 20px; color: #000000; ">3.1 - Jointures des fichiers population et disponibilité alimentaire</h3>
</div>

In [252]:
# Vérifier les correspondances des pays dans chaque dataframe
correspondances_1 = trouver_correspondances_pays(
    df_population_2017["Zone"].tolist(), df_dispo_alimentaire_2017["Zone"].tolist()
)

# Afficher les correspondances
correspondances_1.head()

,Pays_Source,Pays_Match,Score
0,saint-siege,None,86
1,tokelaou,None,54
2,nioue,None,72
3,iles falkland (malvinas),None,86
4,montserrat,None,61


In [253]:
# Rechercher et afficher les pays qui n'ont pas matché
correspondances_1.query("Score < 100")

,Pays_Source,Pays_Match,Score
0,saint-siege,None,86
1,tokelaou,None,54
2,nioue,None,72
3,iles falkland (malvinas),None,86
4,montserrat,None,61
5,saint-pierre-et-miquelon,None,86
6,"sainte-helene, ascension et tristan da cunha",None,86
7,saint-barthelemy,None,86
8,nauru,None,57
9,tuvalu,None,57


In [254]:
# Réaliser la jointure avec Code Zone
jointure_1 = inner_join(
    df_population_2017, df_dispo_alimentaire_2017, on="Code zone", indicator=True
)
jointure_1.head()

Rapport de qualité de la jointure :
|    | Type de jointure   |   Nombre |
|----|--------------------|----------|
|  0 | both               |      172 |
|  1 | left_only          |        0 |
|  2 | right_only         |        0 |


,Code zone,Zone_x,population (millions hab),Zone_y,Disponibilité alimentaire en quantité (kg/personne/an),Importations - Quantité,Production,Exportations - Quantité,Disponibilité intérieure,_merge
0,188,saint-kitts-et-nevis,52045.0,saint-kitts-et-nevis,55.77,4.0,0.0,0.0,4.0,both
1,55,dominique,71458.0,dominique,35.19,4.0,0.0,0.0,4.0,both
2,8,antigua-et-barbuda,95426.0,antigua-et-barbuda,54.10,7.0,0.0,0.0,7.0,both
3,191,saint-vincent-et-les grenadines,109827.0,saint-vincent-et-les grenadines,72.31,9.0,0.0,NaN,8.0,both
4,86,grenade,110874.0,grenade,45.70,7.0,1.0,NaN,8.0,both


In [255]:
# Afficher le descriptif des variables
description_variables(jointure_1)

Description donnée(s):
Le jeu de données comporte: 172 observation(s)
Le jeu de données comporte: 10 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Exportations - Quantité,float64,172,135,50,37,21.51
Production,float64,172,168,125,4,2.33
Importations - Quantité,float64,172,170,84,2,1.16
Disponibilité intérieure,float64,172,170,135,2,1.16
Code zone,int64,172,172,172,0,0.00
Zone_x,object,172,172,172,0,0.00
population (millions hab),float64,172,172,172,0,0.00
Zone_y,object,172,172,172,0,0.00
Disponibilité alimentaire en quantité (kg/personne/an),float64,172,172,169,0,0.00
_merge,category,172,172,1,0,0.00


In [256]:
# Supprimer les colonnes inutiles
jointure_1 = clean_dataframe(

    jointure_1, columns_to_drop=["_merge", "Zone_y"], rename_columns={"Zone_x": "Zone"}
)

In [257]:
# Afficher les lignes qui contiennent des valeurs NaN
# Afficher les lignes qui contiennet des valeurs NaN
jointure_1_nan = jointure_1[jointure_1.isna().any(axis=1)]
print("Lignes avec des NaN :")
jointure_1_nan.head()

Lignes avec des NaN :


,Code zone,Zone,population (millions hab),Disponibilité alimentaire en quantité (kg/personne/an),Importations - Quantité,Production,Exportations - Quantité,Disponibilité intérieure
3,191,saint-vincent-et-les grenadines,109827.0,72.31,9.0,0.0,NaN,8.0
4,86,grenade,110874.0,45.70,7.0,1.0,NaN,8.0
5,83,kiribati,114158.0,17.98,1.0,1.0,NaN,2.0
6,189,sainte-lucie,180954.0,56.69,10.0,1.0,NaN,11.0
8,193,sao tome-et-principe,207089.0,9.47,2.0,1.0,NaN,2.0


In [258]:
# Description du dataframe
description_variables(jointure_1)

Description donnée(s):
Le jeu de données comporte: 172 observation(s)
Le jeu de données comporte: 8 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Exportations - Quantité,float64,172,135,50,37,21.51
Production,float64,172,168,125,4,2.33
Importations - Quantité,float64,172,170,84,2,1.16
Disponibilité intérieure,float64,172,170,135,2,1.16
Code zone,int64,172,172,172,0,0.00
Zone,object,172,172,172,0,0.00
population (millions hab),float64,172,172,172,0,0.00
Disponibilité alimentaire en quantité (kg/personne/an),float64,172,172,169,0,0.00


In [259]:
colonnes_a_convertir = [
    "Exportations - Quantité",
    "Production",
    "Importations - Quantité",
    "Disponibilité intérieure",
    "Disponibilité alimentaire en quantité (kg/personne/an)",
]

# Supprimer les caractères parasites avant conversion
for col in colonnes_a_convertir:
    jointure_1[col] = (
        jointure_1[col]
        .astype(str)  # Convertir en chaîne
        .str.replace(
            r"[^\d.,-]", "", regex=True
        )  # Garder uniquement les chiffres, points et virgules
        .str.replace(",", ".")  # Remplacer les virgules par des points
    )
    jointure_1[col] = pd.to_numeric(
        jointure_1[col], errors="coerce"
    )  # Convertir en numérique

In [260]:
description_variables(jointure_1)

Description donnée(s):
Le jeu de données comporte: 172 observation(s)
Le jeu de données comporte: 8 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Exportations - Quantité,float64,172,135,50,37,21.51
Production,float64,172,168,125,4,2.33
Importations - Quantité,float64,172,170,84,2,1.16
Disponibilité intérieure,float64,172,170,135,2,1.16
Code zone,int64,172,172,172,0,0.00
Zone,object,172,172,172,0,0.00
population (millions hab),float64,172,172,172,0,0.00
Disponibilité alimentaire en quantité (kg/personne/an),float64,172,172,169,0,0.00


<h3 style="margin: auto; padding: 20px; color: #000000; ">3.2 - Jointure des fichiers - Taux de change standardise et PIB par habitant - PPA </h3>
</div>

In [261]:
# Identifier les correspondances des pays dans chaque dataframe
correspondance_2 = trouver_correspondances_pays(
    df_taux_change["Zone"].tolist(),
    df_pib_par_habitant["Zone"].tolist(),
)

# Afficher les correspondances
correspondance_2.head()

,Pays_Source,Pays_Match,Score
0,afghanistan,afghanistan,100
1,afrique du sud,afrique du sud,100
2,albanie,albanie,100
3,algerie,algerie,100
4,allemagne,allemagne,100


In [262]:
# Afficher les pays qui n'ont pas matché (recherche stricte)
correspondance_2_no_matching = correspondance_2.query("Score < 100")
correspondance_2_no_matching.head()

,Pays_Source,Pays_Match,Score
7,anguilla,None,71
12,aruba,None,67
49,cuba,None,51
50,curacao,None,53
58,erythree,None,60


In [263]:
# Nombre de pays sans correspondance
correspondance_2_no_matching.count()

Pays_Source    21
Pays_Match      0
Score          21
dtype: int64

In [264]:
# Renommer les noms des pays pour correspondances - df taux de change
df_taux_change.loc[
    df_taux_change["Zone"] == "venezuela (republique bolivarienne du)", "Zone"
] = "venezuela"

In [265]:
# Réaliser la jointure
jointure_2 = inner_join(
    df_taux_change, df_pib_par_habitant, on="Code zone (M49)", indicator=True
)

Rapport de qualité de la jointure :
|    | Type de jointure   |   Nombre |
|----|--------------------|----------|
|  0 | both               |      189 |
|  1 | left_only          |        0 |
|  2 | right_only         |        0 |


In [266]:
description_variables(jointure_2)

Description donnée(s):
Le jeu de données comporte: 189 observation(s)
Le jeu de données comporte: 6 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Code zone (M49),int64,189,189,189,0,0.0
Zone_x,object,189,189,189,0,0.0
Taux de change,float64,189,189,131,0,0.0
Zone_y,object,189,189,189,0,0.0
PIB par habitant - PPA,float64,189,189,189,0,0.0
_merge,category,189,189,1,0,0.0


In [267]:
# Nettoyer le dataframe
jointure_2 = clean_dataframe(
    jointure_2,
    columns_to_drop=["Zone_y", "Code zone (M49)", "_merge"],
    rename_columns={"Zone_x": "Zone"},
)

In [268]:
jointure_2.head()

,Zone,Taux de change,PIB par habitant - PPA
0,afghanistan,68.026904,2956.8
1,afrique du sud,13.333781,14823.6
2,albanie,119.099667,14229.3
3,algerie,110.973017,13805.4
4,allemagne,0.885206,61563.6


<h3 style="margin: auto; padding: 20px; color: #000000; ">3.3 - Jointure des fichiers - Stabilité Politique et Couts des procédures d'exportations - PPA </h3>
</div>

In [269]:
# Vérifier à nouveau les correspondances
correspondance_3 = trouver_correspondances_pays(
    df_stabilite_politique["Zone"].tolist(),
    df_couts_procedure_exportation["Zone"].tolist(),
)

# Afficher les correspondances
correspondance_3.head()

,Pays_Source,Pays_Match,Score
0,afghanistan,afghanistan,100
1,afrique du sud,afrique du sud,100
2,albanie,albanie,100
3,algerie,algerie,100
4,allemagne,allemagne,100


In [270]:
# Afficher les pays qui n'ont pas matché (recherche stricte)
correspondance_3_no_matching = correspondance_3.query("Score < 100")
correspondance_3_no_matching.head()

,Pays_Source,Pays_Match,Score
23,bielorussie,None,67
24,bolivie (etat plurinational de),None,90
35,cap-vert,None,67
37,chine - ras de hong-kong,chine ras de hong kong,96
38,chine - ras de macao,None,90
39,"chine, continentale",None,90
40,"chine, taiwan province de",None,90
44,congo,None,90
47,croatie (hrvatska),None,90
59,etats-unis d'amerique,None,90


In [271]:
# Nombre de pays sans correspondance
correspondance_3_no_matching.count()

Pays_Source    27
Pays_Match      5
Score          27
dtype: int64

In [354]:
# Renommer les noms des pays pour correspondances - df couts des procedures exportations
df_couts_procedure_exportation.loc[
    df_couts_procedure_exportation["Zone"] == "belarus", "Zone"
] = "bielorussie"

df_couts_procedure_exportation.loc[
    df_couts_procedure_exportation["Zone"] == "cabo verde", "Zone"
] = "cap-vert"

df_couts_procedure_exportation.loc[
    df_couts_procedure_exportation["Zone"] == "iraq", "Zone"
] = "irak"

df_couts_procedure_exportation.loc[
    df_couts_procedure_exportation["Zone"] == "viet nam", "Zone"
] = "vietnam"

df_couts_procedure_exportation.loc[
    df_couts_procedure_exportation["Zone"] == "congo republique democratique du", "Zone"
] = "republique democratique du congo"

df_couts_procedure_exportation.loc[
    df_couts_procedure_exportation["Zone"] == "moldava", "Zone"
] = "republique de moldavie"

df_couts_procedure_exportation.loc[
    df_couts_procedure_exportation["Zone"] == "region administrative speciale de macao",
    "Zone",
] = "chine - ras de macao"

df_couts_procedure_exportation.loc[
    df_couts_procedure_exportation["Zone"] == "chine ras de hong kong",
    "Zone",
] = "chine - ras de hong-kong"

df_couts_procedure_exportation.loc[
    df_couts_procedure_exportation["Zone"] == "chine",
    "Zone",
] = "chine, continentale"

df_couts_procedure_exportation.loc[
    df_couts_procedure_exportation["Zone"] == "republique slovaque", "Zone"
] = "slovaquie"

df_couts_procedure_exportation.loc[
    df_couts_procedure_exportation["Zone"] == "republique tcheque", "Zone"
] = "tchequie"


# Renommer les noms des pays pour correspondances - df stabilite politique

df_stabilite_politique.loc[
    df_stabilite_politique["Zone"] == "bolivie (etat plurinational de)", "Zone"
] = "bolivie"

df_stabilite_politique.loc[
    df_stabilite_politique["Zone"] == "iran (republique islamique d')", "Zone"
] = "iran"

df_stabilite_politique.loc[
    df_stabilite_politique["Zone"] == "venezuela (republique bolivarienne du)", "Zone"
] = "venezuela"

df_stabilite_politique.loc[
    df_stabilite_politique["Zone"] == "croatie (hrvatska)", "Zone"
] = "croatie"

df_stabilite_politique.loc[
    df_stabilite_politique["Zone"]
    == "royaume-uni de grande-bretagne et d'irlande du nord",
    "Zone",
] = "royaume-uni"

df_stabilite_politique.loc[df_stabilite_politique["Zone"] == "salvador", "Zone"] = (
    "el salvador"
)

df_stabilite_politique.loc[
    df_stabilite_politique["Zone"] == "timor oriental", "Zone"
] = "timor-leste"

df_stabilite_politique.loc[
    df_stabilite_politique["Zone"] == "etats-unis d'amerique", "Zone"
] = "etats-unis"

df_stabilite_politique.loc[
    df_stabilite_politique["Zone"] == "iran republique islamique d'", "Zone"
] = "iran"

In [355]:
# Vérifier à nouveau les correspondances
correspondances_4 = trouver_correspondances_pays(
    df_stabilite_politique["Zone"].tolist(),
    df_couts_procedure_exportation["Zone"].tolist(),
)
correspondances_4.head()

,Pays_Source,Pays_Match,Score
0,afghanistan,afghanistan,100
1,afrique du sud,afrique du sud,100
2,albanie,albanie,100
3,algerie,algerie,100
4,allemagne,allemagne,100


In [356]:
# Afficher les pays dont score est inférieur à 100 (recherche stricte)
correspondances_4_no_matching = correspondances_4.query("Score < 100")
correspondances_4_no_matching.head()

,Pays_Source,Pays_Match,Score
40,"chine, taiwan province de",None,86
44,congo,None,90
85,iran,None,90
95,kirghizistan,None,63
116,micronesie (etats federes de),None,90


In [321]:
# Réaliser la jointure
jointure_3 = inner_join(
    df_stabilite_politique, df_couts_procedure_exportation, on="Zone", indicator=True
)

Rapport de qualité de la jointure :
|    | Type de jointure   |   Nombre |
|----|--------------------|----------|
|  0 | both               |      187 |
|  1 | left_only          |        0 |
|  2 | right_only         |        0 |


In [322]:
# Supprimer la colonne _merge
jointure_3 = jointure_3.drop("_merge", axis=1)

In [323]:
# Afficher la description des variables
description_variables(jointure_3)

Description donnée(s):
Le jeu de données comporte: 187 observation(s)
Le jeu de données comporte: 5 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Coûts Procedures d'Exportation,float64,187,176,144,11,5.88
Zone,object,187,187,187,0,0.00
Stabilite Politique,float64,187,187,146,0,0.00
Country Code,object,187,187,187,0,0.00
Indicator Name,object,187,187,3,0,0.00


In [324]:
colonnes_a_convertir = [
    "Coûts Procedures d'Exportation",
    "Stabilite Politique",
]

# Supprimer les caractères parasites avant conversion
for col in colonnes_a_convertir:
    jointure_3[col] = (
        jointure_3[col]
        .astype(str)  # Convertir en chaîne
        .str.replace(
            r"[^\d.,-]", "", regex=True
        )  # Garder uniquement les chiffres, points et virgules
        .str.replace(",", ".")  # Remplacer les virgules par des points
    )
    jointure_3[col] = pd.to_numeric(
        jointure_3[col], errors="coerce"
    )  # Convertir en numérique

In [325]:
# Description du dataframe
description_variables(jointure_3)

Description donnée(s):
Le jeu de données comporte: 187 observation(s)
Le jeu de données comporte: 5 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Coûts Procedures d'Exportation,float64,187,176,144,11,5.88
Zone,object,187,187,187,0,0.00
Stabilite Politique,float64,187,187,146,0,0.00
Country Code,object,187,187,187,0,0.00
Indicator Name,object,187,187,3,0,0.00


<h3 style="margin: auto; padding: 20px; color: #000000; ">3.4 - Jointure des fichiers - Jointure 1 et Jointure 2 </h3>
</div>

In [326]:
# Vérifier à nouveau les correspondances
correspondances_5 = trouver_correspondances_pays(
    jointure_1["Zone"].tolist(),
    jointure_2["Zone"].tolist(),
)

# Afficher les correspondances
correspondances_5.head()

,Pays_Source,Pays_Match,Score
0,saint-kitts-et-nevis,saint-kitts-et-nevis,100
1,dominique,dominique,100
2,antigua-et-barbuda,antigua-et-barbuda,100
3,saint-vincent-et-les grenadines,saint-vincent-et-les grenadines,100
4,grenade,grenade,100


In [327]:
# Afficher les pays qui n'ont pas matché (recherche stricte)
correspondances_5_no_matching = correspondances_5.query("Score < 100")
correspondances_5_no_matching

,Pays_Source,Pays_Match,Score
9,polynesie francaise,None,75
10,nouvelle-caledonie,None,71
66,turkmenistan,None,64
96,cuba,None,51
108,pays-bas,None,90
117,"chine, taiwan province de",None,86
121,republique populaire democratique de coree,None,89
124,yemen,None,54
127,venezuela (republique bolivarienne du),None,86


In [328]:
# Réaliser la jointure
jointure_4 = inner_join(jointure_1, jointure_2, on="Zone", indicator=True)

Rapport de qualité de la jointure :
|    | Type de jointure   |   Nombre |
|----|--------------------|----------|
|  0 | both               |      163 |
|  1 | left_only          |        0 |
|  2 | right_only         |        0 |


In [329]:
description_variables(jointure_4)

Description donnée(s):
Le jeu de données comporte: 163 observation(s)
Le jeu de données comporte: 11 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Exportations - Quantité,float64,163,129,49,34,20.86
Production,float64,163,159,119,4,2.45
Importations - Quantité,float64,163,161,81,2,1.23
Disponibilité intérieure,float64,163,161,128,2,1.23
Code zone,int64,163,163,163,0,0.00
Zone,object,163,163,163,0,0.00
population (millions hab),float64,163,163,163,0,0.00
Disponibilité alimentaire en quantité (kg/personne/an),float64,163,163,160,0,0.00
Taux de change,float64,163,163,117,0,0.00
PIB par habitant - PPA,float64,163,163,163,0,0.00


In [330]:
# Nettoyer le dataframe
jointure_4 = clean_dataframe(
    jointure_4,
    columns_to_drop=["_merge"],
    rename_columns=None,
)

# Afficher le dataframe
jointure_4.head()

,Code zone,Zone,population (millions hab),Disponibilité alimentaire en quantité (kg/personne/an),Importations - Quantité,Production,Exportations - Quantité,Disponibilité intérieure,Taux de change,PIB par habitant - PPA
0,188,saint-kitts-et-nevis,52045.0,55.77,4.0,0.0,0.0,4.0,2.7,29272.1
1,55,dominique,71458.0,35.19,4.0,0.0,0.0,4.0,2.7,15285.2
2,8,antigua-et-barbuda,95426.0,54.10,7.0,0.0,0.0,7.0,2.7,26949.1
3,191,saint-vincent-et-les grenadines,109827.0,72.31,9.0,0.0,NaN,8.0,2.7,15374.6
4,86,grenade,110874.0,45.70,7.0,1.0,NaN,8.0,2.7,15562.3


In [331]:
colonnes_a_convertir = [
    "Taux de change",
    "PIB par habitant - PPA",
]

# Supprimer les caractères parasites avant conversion
for col in colonnes_a_convertir:
    jointure_4[col] = (
        jointure_4[col]
        .astype(str)  # Convertir en chaîne
        .str.replace(
            r"[^\d.,-]", "", regex=True
        )  # Garder uniquement les chiffres, points et virgules
        .str.replace(",", ".")  # Remplacer les virgules par des points
    )
    jointure_4[col] = pd.to_numeric(
        jointure_4[col], errors="coerce"
    )  # Convertir en numérique

In [332]:
# Afficher la description du dataframe
description_variables(jointure_4)

Description donnée(s):
Le jeu de données comporte: 163 observation(s)
Le jeu de données comporte: 10 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Exportations - Quantité,float64,163,129,49,34,20.86
Production,float64,163,159,119,4,2.45
Importations - Quantité,float64,163,161,81,2,1.23
Disponibilité intérieure,float64,163,161,128,2,1.23
Code zone,int64,163,163,163,0,0.00
Zone,object,163,163,163,0,0.00
population (millions hab),float64,163,163,163,0,0.00
Disponibilité alimentaire en quantité (kg/personne/an),float64,163,163,160,0,0.00
Taux de change,float64,163,163,117,0,0.00
PIB par habitant - PPA,float64,163,163,163,0,0.00


<h3 style="margin: auto; padding: 20px; color: #000000; ">3.5 - Jointure des fichiers - Jointure 3 et Jointure 4 </h3>
</div>

In [333]:
# Vérifier à nouveau les correspondances
correspondances_6 = trouver_correspondances_pays(
    jointure_3["Zone"].tolist(),
    jointure_4["Zone"].tolist(),
)
correspondances_6.head()

,Pays_Source,Pays_Match,Score
0,afghanistan,afghanistan,100
1,afrique du sud,afrique du sud,100
2,albanie,albanie,100
3,algerie,algerie,100
4,allemagne,allemagne,100


In [334]:
# Afficher les pays qui n'ont pas matché (recherche stricte)
correspondances_6_no_matching = correspondances_6.query("Score < 100")
correspondances_6_no_matching.head()

,Pays_Source,Pays_Match,Score
5,andorre,None,66
15,bahrein,None,67
21,bermudes,None,67
22,bhoutan,None,67
23,bielorussie,None,67


In [335]:
# Nombre de pays qui n'ont pas matché
correspondances_6_no_matching.count()

Pays_Source    38
Pays_Match      0
Score          38
dtype: int64

In [336]:
# Dictionnaire des correspondances
remplacement_pays = {
    "belarus": "bielorussie",
    "cabo verde": "cap-vert",
    "iraq": "irak",
    "viet nam": "vietnam",
    "etats-unis d'amerique": "etats-unis",
    "congo": "republique democratique du congo",
    "royaume-uni de grande-bretagne et d'irlande du nord": "royaume-uni",
}

# Appliquer les remplacements
# jointure_8["Zone"] = jointure_8["Zone"].replace(remplacement_pays)

jointure_4["Zone"] = jointure_4["Zone"].replace(remplacement_pays)

In [337]:
# Vérifier à nouveau les correspondances
correspondances_7 = trouver_correspondances_pays(
    jointure_3["Zone"].tolist(),
    jointure_4["Zone"].tolist(),
)
correspondances_7.head()

,Pays_Source,Pays_Match,Score
0,afghanistan,afghanistan,100
1,afrique du sud,afrique du sud,100
2,albanie,albanie,100
3,algerie,algerie,100
4,allemagne,allemagne,100


In [338]:
# Afficher les pays qui n'ont pas matché (recherche stricte)
correspondances_7_no_matching = correspondances_7.query("Score < 100")
correspondances_7_no_matching.head()

,Pays_Source,Pays_Match,Score
5,andorre,None,66
15,bahrein,None,67
21,bermudes,None,57
22,bhoutan,None,67
24,bolivie,None,90


In [339]:
# Réaliser la jointure
jointure_5 = inner_join(jointure_3, jointure_4, on="Zone", indicator=True)

Rapport de qualité de la jointure :
|    | Type de jointure   |   Nombre |
|----|--------------------|----------|
|  0 | both               |      156 |
|  1 | left_only          |        0 |
|  2 | right_only         |        0 |


In [340]:
# Description du dataframe
description_variables(jointure_5)

Description donnée(s):
Le jeu de données comporte: 156 observation(s)
Le jeu de données comporte: 15 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Exportations - Quantité,float64,156,124,49,32,20.51
Production,float64,156,152,114,4,2.56
Importations - Quantité,float64,156,154,80,2,1.28
Disponibilité intérieure,float64,156,154,124,2,1.28
Coûts Procedures d'Exportation,float64,156,155,126,1,0.64
Zone,object,156,156,156,0,0.00
Stabilite Politique,float64,156,156,127,0,0.00
Country Code,object,156,156,156,0,0.00
Indicator Name,object,156,156,2,0,0.00
Code zone,int64,156,156,156,0,0.00


In [341]:
# Supprimer les données manquantes
jointure_5 = handle_missing_values(jointure_5, strategy="drop")


jointure_5.head()

,Zone,Stabilite Politique,Country Code,Indicator Name,Coûts Procedures d'Exportation,Code zone,population (millions hab),Disponibilité alimentaire en quantité (kg/personne/an),Importations - Quantité,Production,Exportations - Quantité,Disponibilité intérieure,Taux de change,PIB par habitant - PPA,_merge
1,afrique du sud,-0.28,ZAF,"Coût des procédures d’exportation, formalités ...",1257.00,202,57009756.0,35.69,514.0,1667.0,63.0,2118.0,13.333781,14823.6,both
2,albanie,0.38,ALB,"Coût des procédures d’exportation, formalités ...",54.67,3,2884169.0,16.36,38.0,13.0,0.0,47.0,119.099667,14229.3,both
3,algerie,-0.92,DZA,"Coût des procédures d’exportation, formalités ...",592.89,4,41389189.0,6.38,2.0,275.0,0.0,277.0,110.973017,13805.4,both
4,allemagne,0.59,DEU,"Coût des procédures d’exportation, formalités ...",345.00,79,82658409.0,19.47,842.0,1514.0,646.0,1739.0,0.885206,61563.6,both
5,angola,-0.33,AGO,"Coût des procédures d’exportation, formalités ...",825.00,7,29816766.0,10.56,277.0,42.0,0.0,319.0,165.915951,9050.0,both


In [342]:
# Nettoyer le dataframe
jointure_5 = clean_dataframe(
    jointure_5,
    columns_to_drop=["_merge", "Indicator Name"],
    rename_columns=None,
)

# Afficher le dataframe
jointure_5.head()

,Zone,Stabilite Politique,Country Code,Coûts Procedures d'Exportation,Code zone,population (millions hab),Disponibilité alimentaire en quantité (kg/personne/an),Importations - Quantité,Production,Exportations - Quantité,Disponibilité intérieure,Taux de change,PIB par habitant - PPA
1,afrique du sud,-0.28,ZAF,1257.00,202,57009756.0,35.69,514.0,1667.0,63.0,2118.0,13.333781,14823.6
2,albanie,0.38,ALB,54.67,3,2884169.0,16.36,38.0,13.0,0.0,47.0,119.099667,14229.3
3,algerie,-0.92,DZA,592.89,4,41389189.0,6.38,2.0,275.0,0.0,277.0,110.973017,13805.4
4,allemagne,0.59,DEU,345.00,79,82658409.0,19.47,842.0,1514.0,646.0,1739.0,0.885206,61563.6
5,angola,-0.33,AGO,825.00,7,29816766.0,10.56,277.0,42.0,0.0,319.0,165.915951,9050.0


In [343]:
# Description du dataframe
description_variables(jointure_5)

Description donnée(s):
Le jeu de données comporte: 123 observation(s)
Le jeu de données comporte: 13 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Zone,object,123,123,123,0,0.0
Stabilite Politique,float64,123,123,100,0,0.0
Country Code,object,123,123,123,0,0.0
Coûts Procedures d'Exportation,float64,123,123,97,0,0.0
Code zone,int64,123,123,123,0,0.0
population (millions hab),float64,123,123,123,0,0.0
Disponibilité alimentaire en quantité (kg/personne/an),float64,123,123,122,0,0.0
Importations - Quantité,float64,123,123,78,0,0.0
Production,float64,123,123,103,0,0.0
Exportations - Quantité,float64,123,123,49,0,0.0


<div style="background-color: #000000;" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 4 - Feature Engineering - Création de 3 nouvelles variables</h2>
</div>

<h4 style="margin: auto; padding: 20px; color: #000000; "> 4.1.1 Indice de dépendance alimentaire étrangère </h4>
</div>

 L'objectif est de comparer les importations avec la production nationale

- Identifier les marchés où la production locale est faible comparée aux importations. 
- Cibler les pays où la production locale est peu compétitive face aux importations.

In [344]:
jointure_5["Indice Dependance Alimentaire etrangere"] = jointure_5[
    "Importations - Quantité"
] / (jointure_5["Importations - Quantité"] + jointure_5["Production"])

# Afficher le dataframe
jointure_5.head()

,Zone,Stabilite Politique,Country Code,Coûts Procedures d'Exportation,Code zone,population (millions hab),Disponibilité alimentaire en quantité (kg/personne/an),Importations - Quantité,Production,Exportations - Quantité,Disponibilité intérieure,Taux de change,PIB par habitant - PPA,Indice Dependance Alimentaire etrangere
1,afrique du sud,-0.28,ZAF,1257.00,202,57009756.0,35.69,514.0,1667.0,63.0,2118.0,13.333781,14823.6,0.235672
2,albanie,0.38,ALB,54.67,3,2884169.0,16.36,38.0,13.0,0.0,47.0,119.099667,14229.3,0.745098
3,algerie,-0.92,DZA,592.89,4,41389189.0,6.38,2.0,275.0,0.0,277.0,110.973017,13805.4,0.007220
4,allemagne,0.59,DEU,345.00,79,82658409.0,19.47,842.0,1514.0,646.0,1739.0,0.885206,61563.6,0.357385
5,angola,-0.33,AGO,825.00,7,29816766.0,10.56,277.0,42.0,0.0,319.0,165.915951,9050.0,0.868339


<h4 style="margin: auto; padding: 20px; color: #000000; "> 4.1.2 Balance commerciale alimentaire</h4>
</div>

Calcule la différence entre les exportations et les importations pour mesurer si un pays est un exportateur net ou un importateur net d'aliments

In [345]:
jointure_5["Balance Commerciale"] = (
    jointure_5["Exportations - Quantité"] - jointure_5["Importations - Quantité"]
)

# Afficher le dataframe
jointure_5.head()

,Zone,Stabilite Politique,Country Code,Coûts Procedures d'Exportation,Code zone,population (millions hab),Disponibilité alimentaire en quantité (kg/personne/an),Importations - Quantité,Production,Exportations - Quantité,Disponibilité intérieure,Taux de change,PIB par habitant - PPA,Indice Dependance Alimentaire etrangere,Balance Commerciale
1,afrique du sud,-0.28,ZAF,1257.00,202,57009756.0,35.69,514.0,1667.0,63.0,2118.0,13.333781,14823.6,0.235672,-451.0
2,albanie,0.38,ALB,54.67,3,2884169.0,16.36,38.0,13.0,0.0,47.0,119.099667,14229.3,0.745098,-38.0
3,algerie,-0.92,DZA,592.89,4,41389189.0,6.38,2.0,275.0,0.0,277.0,110.973017,13805.4,0.007220,-2.0
4,allemagne,0.59,DEU,345.00,79,82658409.0,19.47,842.0,1514.0,646.0,1739.0,0.885206,61563.6,0.357385,-196.0
5,angola,-0.33,AGO,825.00,7,29816766.0,10.56,277.0,42.0,0.0,319.0,165.915951,9050.0,0.868339,-277.0


<h4 style="margin: auto; padding: 20px; color: #000000; "> 4.1.3 Indice de dépendance alimentaire</h4>
</div>

L'objectif est de mesurer la dépendance aux importations pour satisfaire la demande locale
- Identifier des pays où la dépendance aux importations est élevée.

In [346]:
# Calculer l'indice de dépendance alimentaire
jointure_5["Dependance Alimentaire"] = (
    jointure_5["Importations - Quantité"] / jointure_5["Disponibilité intérieure"]
)

# Afficher le dataframe
jointure_5.head()

,Zone,Stabilite Politique,Country Code,Coûts Procedures d'Exportation,Code zone,population (millions hab),Disponibilité alimentaire en quantité (kg/personne/an),Importations - Quantité,Production,Exportations - Quantité,Disponibilité intérieure,Taux de change,PIB par habitant - PPA,Indice Dependance Alimentaire etrangere,Balance Commerciale,Dependance Alimentaire
1,afrique du sud,-0.28,ZAF,1257.00,202,57009756.0,35.69,514.0,1667.0,63.0,2118.0,13.333781,14823.6,0.235672,-451.0,0.242682
2,albanie,0.38,ALB,54.67,3,2884169.0,16.36,38.0,13.0,0.0,47.0,119.099667,14229.3,0.745098,-38.0,0.808511
3,algerie,-0.92,DZA,592.89,4,41389189.0,6.38,2.0,275.0,0.0,277.0,110.973017,13805.4,0.007220,-2.0,0.007220
4,allemagne,0.59,DEU,345.00,79,82658409.0,19.47,842.0,1514.0,646.0,1739.0,0.885206,61563.6,0.357385,-196.0,0.484186
5,angola,-0.33,AGO,825.00,7,29816766.0,10.56,277.0,42.0,0.0,319.0,165.915951,9050.0,0.868339,-277.0,0.868339


In [347]:
# Description du jeu de données
description_variables(jointure_5)

Description donnée(s):
Le jeu de données comporte: 123 observation(s)
Le jeu de données comporte: 16 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Zone,object,123,123,123,0,0.0
Stabilite Politique,float64,123,123,100,0,0.0
Country Code,object,123,123,123,0,0.0
Coûts Procedures d'Exportation,float64,123,123,97,0,0.0
Code zone,int64,123,123,123,0,0.0
population (millions hab),float64,123,123,123,0,0.0
Disponibilité alimentaire en quantité (kg/personne/an),float64,123,123,122,0,0.0
Importations - Quantité,float64,123,123,78,0,0.0
Production,float64,123,123,103,0,0.0
Exportations - Quantité,float64,123,123,49,0,0.0


<div style="background-color: #000000;" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 5 - Valider les contraintes du cahier des charges</h2>
</div>

Minimum 100 pays et 60% de la population mondiale

In [350]:
# Description du jeu de données
description_variables(jointure_5)

Description donnée(s):
Le jeu de données comporte: 123 observation(s)
Le jeu de données comporte: 16 colonne(s)


,type,total,n,unique,NaN,pourcentage_(NaN)
Zone,object,123,123,123,0,0.0
Stabilite Politique,float64,123,123,100,0,0.0
Country Code,object,123,123,123,0,0.0
Coûts Procedures d'Exportation,float64,123,123,97,0,0.0
Code zone,int64,123,123,123,0,0.0
population (millions hab),float64,123,123,123,0,0.0
Disponibilité alimentaire en quantité (kg/personne/an),float64,123,123,122,0,0.0
Importations - Quantité,float64,123,123,78,0,0.0
Production,float64,123,123,103,0,0.0
Exportations - Quantité,float64,123,123,49,0,0.0


In [351]:
# Population totale (France incluse)
x = jointure_5["population (millions hab)"].sum()

# Population totale (France incluse)
y = df_population_2017["population (millions hab)"].sum()

# Pourcentage de la population mondiale (France incluse)
x / y

0.850159162202517

In [352]:
# Exporter et renommer le fichier au format .csv
jointure_5.to_csv("df_poule_qui_chante.csv", index=False)